In [1]:
import torch
import time

Installing the required libraries

In [2]:
pip uninstall -y transformers

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2


In [3]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 62.4 MB/s eta 0:00:00


In [4]:
pip install tensorflow --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [5]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9414aa6d834b906cdc4ba22d802982e11234f3306e12805970dca9f9e666d40e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [6]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [7]:
pip install datasets

Newer version of peft are generate adapter_model.safetensors instead of adapter_model.bin, which is not suitable for loading the trained model in most systems, so I am uninstalling the old peft module if it exists and downloading the 0.6.2 version

In [8]:
pip uninstall -y peft

In [9]:
pip install peft==0.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 14.2 MB/s eta 0:00:00


Some systems have an older version of accelerate installed, so I am uninstalling it and reinstalling the newer version below

In [10]:
pip uninstall -y accelerate

Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2


In [11]:
pip install accelerate

  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


Loading the base model and creating a tokenizer from the AutoTokenizer object

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer

model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# NOTE:
In the **tokenize function**, it is advised to name the input ids as **'input_ids'** and output or the label as **'labels'** because the model is able to process only these two parameters, and it gave me errors when I try to name it something else, but this may not be same issue in the future, as there may by updates.

In [13]:
def tokenize_function(example):
    start_prompt = 'Summarize the following\n'
    end_prompt = "\n Answer:"
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["output"]]

    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True,
                                     return_tensors='pt').input_ids
    example['labels'] = tokenizer(example['instruction'], padding="max_length", truncation=True,
                                  return_tensors='pt').input_ids



    return example

In [14]:
from datasets import load_dataset

In [15]:
dataset = load_dataset("causal-lm/finance")

Generating train split:   0%|          | 0/62020 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6892 [00:00<?, ? examples/s]

In [91]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 62020
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 6892
    })
})

In [92]:
dataset = dataset.remove_columns(['input'])

In [17]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/62020 [00:00<?, ? examples/s]

Map:   0%|          | 0/6892 [00:00<?, ? examples/s]

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'input_ids', 'labels'],
        num_rows: 62020
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'input_ids', 'labels'],
        num_rows: 6892
    })
})

In [19]:
tokenized_datasets = tokenized_datasets.remove_columns(['instruction', 'output'])

In [76]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=64, # Rank
    lora_alpha=64,# this means that (64/64=1) 100% of the target modules are selected and fine tuned
    target_modules=["q", "v"], # q and v are the important attention layers to train on, this information can be extracted by printing the pre trained model
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)


In [77]:
peft_model = get_peft_model(original_model,
                            lora_config)

In [78]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [79]:
from transformers import TrainingArguments, Trainer

In [80]:
output_dir = f'/content/gdrive/MyDrive/finance_peft_training -{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=6, # if the batch size is too big, then 'cuda out of memory error' may arise for machines with small memory
    learning_rate=1e-3,
    num_train_epochs=40,
    logging_steps=1,
    max_steps=40,
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [81]:
peft_trainer.train()

Step,Training Loss
1,40.250000
2,38.250000
3,36.000000
4,31.250000
5,29.125000
6,26.375000
7,23.125000
8,19.625000
9,17.000000
10,13.875000


TrainOutput(global_step=40, training_loss=10.230859375, metrics={'train_runtime': 74.9935, 'train_samples_per_second': 3.2, 'train_steps_per_second': 0.533, 'total_flos': 169560149852160.0, 'train_loss': 10.230859375, 'epoch': 0.0})

Saving the Model locally

In [82]:
peft_model_path="/content/gdrive/MyDrive/peft-config3"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('/content/gdrive/MyDrive/peft-config3/tokenizer_config.json',
 '/content/gdrive/MyDrive/peft-config3/special_tokens_map.json',
 '/content/gdrive/MyDrive/peft-config3/spiece.model',
 '/content/gdrive/MyDrive/peft-config3/added_tokens.json',
 '/content/gdrive/MyDrive/peft-config3/tokenizer.json')

Loading and running the original model and peft trained model in cpu

In [83]:
from peft import PeftModel, PeftConfig


model_name = 'google/flan-t5-base'

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       '/content/gdrive/MyDrive/peft-config3',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [84]:
from transformers import GenerationConfig

In [85]:
peft_model = peft_model.to('cpu')
original_model = original_model.to('cpu')

In [86]:
content = """The market is currently navigating through a phase of uncertainty.
Despite a persistent dominance by growth trades, overall economic growth expectations remain subdued.
The forthcoming economic data, particularly jobless claims and nonfarm payroll figures, are anticipated to be critical in shaping market sentiments."""


In [90]:
prompt = f"""
Summarize the following\n

{content}

\nAnswer:"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(min_length=50, max_length=200, num_beams=1, do_sample=True, temperature=0.5))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

print('--------------------------------------------------')
print(f'Baseline content:\n{content}')
print('--------------------------------------------------')


peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(min_length=50, max_length=200, num_beams=1, do_sample=True, temperature=0.5))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)


print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print('--------------------------------------------------')
print(f'PEFT MODEL:\n {peft_model_text_output}')

--------------------------------------------------
Baseline content:
The market is currently navigating through a phase of uncertainty. 
Despite a persistent dominance by growth trades, overall economic growth expectations remain subdued. 
The forthcoming economic data, particularly jobless claims and nonfarm payroll figures, are anticipated to be critical in shaping market sentiments.
--------------------------------------------------
ORIGINAL MODEL:
Collections: Consumers - Forecast:
--------------------------------------------------
PEFT MODEL:
 This is a key time to make sure you are prepared to assess the state of the market.
